In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from collections import defaultdict
import math
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib notebook
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.palettes import Category20
output_notebook()

Loading BokehJS ...

In [2]:
def init_graph(G,node_adj_frame):
    G.add_nodes_from([i for i in range(len(node_adj_frame))])
    labels = {}
    labels = node_adj_frame.columns
    for i in range(len(node_adj_frame)):
        snode = node_adj_frame[labels[0]][i]-1
        temp = node_adj_frame[labels[2]][i]
        if ',' in str(temp):
            sedge_arr = temp.split(',')
            for j in range(0, len(sedge_arr)):
                k = int(sedge_arr[j])
                G.add_edge(snode, k-1)
        elif np.isnan(temp):
            print("ERROR: Not found in the adjacency excel sheet")
        else:
            G.add_edge(snode, int(temp)-1)
    return

In [3]:
def init_graph_attr(G, AdjFile, df, columns):
    node_adj_frame = pd.read_excel(AdjFile)
    node_list = node_adj_frame["District_Name"].tolist()
    node_list.insert(80, "")
    nodeAttr = {}
    init_graph(G, node_adj_frame)
    
    capability_vector = list(zip(*(df[col] for col in columns)))
    node_attri_dict = dict(zip(df["District"], capability_vector))
    node_attri_dict = dict((k, v) for k, v in node_attri_dict.items())

    for i in range(len(node_adj_frame)):
        temp = {}
        temp["capabilityvector"] = node_attri_dict[node_list[i]]
        temp["nodeStress"] = 0
        temp["name"] = node_list[i]
        nodeAttr[i] = temp

    nx.set_node_attributes(G, nodeAttr)

In [4]:
G = nx.Graph()
df = pd.read_excel('KAG 2016-17/Agriculture/Agriculture_KAG_2016_17.xlsx')
adjacency_file = 'Karnataka_District_Adjacency_File.xlsx'

# existing_data = pd.DataFrame(df['District'])

existing_data = pd.read_csv('result.csv')


In [5]:
existing_data = pd.DataFrame(existing_data['District'])

In [6]:
def addList(l1,l2):
    for i in range(len(l1)):
        l1[i] = l1[i] + l2[i]
    return l1

def divList(l1,k):
    for i in range(len(l1)):
        l1[i] = l1[i]/k
    return l1
def l2_normalization(l1,l2):
    k = 0
    for i in range(len(l1)):
        k+= (l1[i] - l2[i])**2
    return math.sqrt(k)

In [7]:
def get_node_stress(G,dim):
    taluka_stress_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStress"] = l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            taluka_stress_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return taluka_stress_dict

In [8]:
def get_node_stability(G,dim):
    taluka_stress_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStress"] = 1 - l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            taluka_stress_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return taluka_stress_dict

In [9]:
columns = ["Rice_Production_223"]
dim = len(columns)
init_graph_attr(G, "KAG 2016-17/Karnataka_District_Adjacency_File.xlsx", df, columns)

# initialstress = get_node_stress(G,dim)
# df["Initial Stress"] = df["District"].map(initialstress)

In [10]:
# df["Initial Stress"]

In [11]:
# def calculate_impact_score(df, base_column, capability_vector, intervention):
#     # Perform simple linear regression
#     X = sm.add_constant(df[base_column])
#     y = df[capability_vector]
#     model = sm.OLS(y, X).fit()

#     # Get coefficients and intercept
#     m, c = model.params[base_column], model.params['const']
#     print(c)
#     # Predicted change in capability_vector for base_column + change_percentage% and base_column - change_percentage%
#     base_column_increase = (1 + intervention/100) * df[base_column]
#     base_column_decrease = (1 - intervention/100) * df[base_column]

#     rp_new_plus = m * base_column_increase + c 
#     rp_new_minus = m * base_column_decrease + c 

#     vector_plus = rp_new_plus - m * df[base_column] - c  
#     vector_minus = rp_new_minus - m * df[base_column] - c

#     # Generate normalized values using MinMaxScaler
#     scaler = MinMaxScaler()

#     normalized_vector_plus = scaler.fit_transform(vector_plus.values.reshape(-1, 1))
#     normalized_vector_minus = scaler.fit_transform(vector_minus.values.reshape(-1, 1))

#     # Create new DataFrame with appropriate column names
#     result_df = pd.DataFrame({
#         f'{capability_vector} ({base_column} +{intervention}%)': rp_new_plus,
#         f'{capability_vector} ({base_column} -{intervention}%)': rp_new_minus,
#         f'Normalized {capability_vector} ({base_column} +{intervention}%)': normalized_vector_plus.flatten(),
#         f'Normalized {capability_vector} ({base_column} -{intervention}%)': normalized_vector_minus.flatten()
#     })

#     return result_df

In [12]:
# base_Column = "TotalNPK_315"
# CapabilityVector = "Jowar_Yield_278"
# change_percentage = 20

# result_df = calculate_impact_score(df, base_Column, CapabilityVector, change_percentage)

# result_df.head()

In [13]:
# def merge_columns_into_dataframe(result_df, existing_data):
#     for column in result_df.columns:
#         existing_data[column] = result_df[column]
#     return existing_data

# # Assuming existing_data is already defined
# existing_data = merge_columns_into_dataframe(result_df, existing_data)
# existing_data.head()

In [14]:
# result_df.head()

In [15]:
existing_data.head()

,District
0,BENGALURU
1,BENGALURU(R)
2,RAMANAGARA
3,CHITRADURGA
4,DAVANAGERE


In [16]:
dim = 1

In [17]:
def calculate_and_map_stability(G, existing_data, adjacency_file, columns_to_pass, change_percentage, dim):
    # Initialize the graph attributes
    init_graph_attr(G, adjacency_file, existing_data, columns_to_pass)

    # Calculate node stress for the given change percentage
    NPK_stability = get_node_stability(G, dim)

    # Create a new column in the result DataFrame
    stability_column_name = f"New Stability(NPK {'+' if change_percentage >= 0 else '-'} {abs(change_percentage)}%)"
    stability_column = existing_data["District"].map(NPK_stability)

    return stability_column, stability_column_name

# Example Usage:
# columns_to_pass_minus20 = ["Normalized Rice_Production_223 (TotalNPK_315 -20%)"]
# columns_to_pass_plus20 = ["Normalized Rice_Production_223 (TotalNPK_315 +20%)"]

# # Calculate and get stress column for NPK -20%
# stability_column_minus20, stability_column_name_minus20 = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass_minus20, -20, dim)

# # Calculate and get stress column for NPK +20%
# stability_column_plus20, stability_column_name_plus20 = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass_plus20, 20, dim)

# # Add stress columns to the result DataFrame
# existing_data[stability_column_name_plus20] = stability_column_plus20

# # existing_data[stability_column_name_minus20] = stability_column_minus20

# # Display the resulting DataFrame
# existing_data.head()

## 1D Capability Vector

In [18]:
# from bokeh.models import Span, Label
# from bokeh.plotting import figure, show, output_notebook
# from bokeh.models.sources import ColumnDataSource
# from bokeh.models.tools import HoverTool
# import statsmodels.api as sm
# from sklearn.preprocessing import MinMaxScaler
# import plotly.express as px

# # Define the function to calculate and visualize impact stability
# def calc_and_vis_impact_stability_1D(G, existing_data, adjacency_file, base_column, capability_vector, intervention, dim):
#     # Calculate Impact Score
#     X = sm.add_constant(df[base_column])
#     y = df[capability_vector]
#     model = sm.OLS(y, X).fit()
    
#     m, c = model.params[base_column], model.params['const']

#     base_column_increase = (1 + intervention/100) * df[base_column]
#     base_column_decrease = (1 - intervention/100) * df[base_column]

#     new_vector_increase = m * base_column_increase + c
#     new_vector_decrease = m * base_column_decrease + c
    
#     vector_plus = new_vector_increase - m * df[base_column] - c  
#     vector_minus = new_vector_decrease - m * df[base_column] - c

#     scaler = MinMaxScaler()
#     normalized_change_increase_vector = scaler.fit_transform(vector_plus.values.reshape(-1, 1))
    
#     scaler2 = MinMaxScaler()
#     normalized_change_decrease_vector = scaler2.fit_transform(vector_minus.values.reshape(-1, 1))
    
    
#     scaler3 = MinMaxScaler()
#     normalized_new_increase_vector = scaler3.fit_transform(new_vector_increase.values.reshape(-1, 1))
    
#     scaler4 = MinMaxScaler()
#     normalized_new_decrease_vector = scaler4.fit_transform(new_vector_decrease.values.reshape(-1, 1))

#     result_df = pd.DataFrame({
#         f'{capability_vector} ({base_column} +{intervention}%)': new_vector_increase,
#         f'{capability_vector} ({base_column} -{intervention}%)': new_vector_decrease,
#         f'Normalized {capability_vector} ({base_column} +{intervention}%)': normalized_new_increase_vector.flatten(),
#         f'Normalized {capability_vector} ({base_column} -{intervention}%)': normalized_new_decrease_vector.flatten(),
#         f'Impact Score {capability_vector} ({base_column} +{intervention}%)': normalized_change_increase_vector.flatten(),
#         f'Impact Score {capability_vector} ({base_column} -{intervention}%)': normalized_change_decrease_vector.flatten(),
#     })
    
#     # Merge Columns
#     for column in result_df.columns:
#         existing_data[column] = result_df[column]

#     # Calculate Stability and add to DataFrame
#     columns_to_pass = [f"Normalized {capability_vector} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"]
#     stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass, intervention, dim)

#     existing_data[stability_column_name] = stability_column

#     # Calculate Stability for the opposite intervention and add to DataFrame
#     opposite_intervention = -intervention
#     columns_to_pass_opposite = [f"Normalized {capability_vector} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"]
#     stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass_opposite, opposite_intervention, dim)

#     existing_data[stability_column_name_opposite] = stability_column_opposite

#     columns_to_pass = [f"Normalized {capability_vector} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"]
#     stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass, intervention, dim)

#     existing_data[stability_column_name] = stability_column

#     # Calculate Stability for the opposite intervention and add to DataFrame
#     opposite_intervention = -intervention
#     columns_to_pass_opposite = [f"Normalized {capability_vector} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"]
#     stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, columns_to_pass_opposite, opposite_intervention, dim)

#     existing_data[stability_column_name_opposite] = stability_column_opposite

#     # Visualize with Plotly for +intervention
#     fig_increase = px.scatter(existing_data,
#                               x=stability_column_name,
#                               y=f'Normalized {capability_vector} ({base_column} +{intervention}%)',
#                               title=f'Impact vs Stability ({capability_vector} - {base_column} +{intervention}%)',
#                               labels={'x': f'Stability ({base_column} +{intervention}%)', 'y': f'Impact ({capability_vector})'},
#                               size_max=8,
#                               width=800,
#                               height=500)
    
#     fig_increase.update_traces(hoverlabel=dict(bgcolor='grey', font=dict(color='white')))

#     # Calculate average values from the DataFrame for +intervention
#     avg_x_increase = existing_data[stability_column_name].mean()
#     avg_y_increase = existing_data[f'Normalized {capability_vector} ({base_column} +{intervention}%)'].mean()

#     # Add average lines for +intervention
#     fig_increase.add_shape(
#         type='line',
#         x0=avg_x_increase,
#         x1=avg_x_increase,
#         y0=fig_increase.data[0].y.min(),
#         y1=fig_increase.data[0].y.max(),
#         line=dict(color='red', width=2)
#     )
#     fig_increase.add_shape(
#         type='line',
#         x0=fig_increase.data[0].x.min(),
#         x1=fig_increase.data[0].x.max(),
#         y0=avg_y_increase,
#         y1=avg_y_increase,
#         line=dict(color='blue', width=2)
#     )

#     # Add labels for average lines for +intervention
#     fig_increase.add_annotation(
#         x=avg_x_increase,
#         y=fig_increase.data[0].y.max(),
#         text=f'Avg {stability_column_name}',
#         showarrow=True,
#         arrowhead=4,
#         arrowcolor='red',
#         ax=0,
#         ay=-40,
#         font=dict(color='red')
#     )
#     fig_increase.add_annotation(
#         x=fig_increase.data[0].x.max(),
#         y=avg_y_increase,
#         text=f'Avg Normalized {capability_vector}',
#         showarrow=True,
#         arrowhead=4,
#         arrowcolor='blue',
#         ax=-40,
#         ay=0,
#         font=dict(color='blue')
#     )

#     # Add tooltips for +intervention
#     fig_increase.update_traces(hoverinfo='text+name',
#                                hovertext=["District: " + str(d) +
#                                           f"<br>Impact ({capability_vector}): {y} <br>Stability ({base_column} +{intervention}%): {x}"
#                                           for x, y, d in zip(fig_increase.data[0].x, fig_increase.data[0].y, existing_data['District'])])

#     # Visualize with Plotly for -intervention
#     fig_decrease = px.scatter(existing_data,
#                               x=stability_column_name_opposite,
#                               y=f'Normalized {capability_vector} ({base_column} -{intervention}%)',
#                               title=f'Impact vs Stability ({capability_vector} - {base_column} -{intervention}%)',
#                               labels={'x': f'Stability ({base_column} -{intervention}%)', 'y': f'Impact ({capability_vector})'},
#                               size_max=8,
#                               width=800,
#                               height=500)

#     # Calculate average values from the DataFrame for -intervention
#     avg_x_decrease = existing_data[stability_column_name_opposite].mean()
#     avg_y_decrease = existing_data[f'Normalized {capability_vector} ({base_column} -{intervention}%)'].mean()

#     # Add average lines for -intervention
#     fig_decrease.add_shape(
#         type='line',
#         x0=avg_x_decrease,
#         x1=avg_x_decrease,
#         y0=fig_decrease.data[0].y.min(),
#         y1=fig_decrease.data[0].y.max(),
#         line=dict(color='red', width=2)
#     )
#     fig_decrease.add_shape(
#         type='line',
#         x0=fig_decrease.data[0].x.min(),
#         x1=fig_decrease.data[0].x.max(),
#         y0=avg_y_decrease,
#         y1=avg_y_decrease,
#         line=dict(color='blue', width=2)
#     )

#     # Add labels for average lines for -intervention
#     fig_decrease.add_annotation(
#         x=avg_x_decrease,
#         y=fig_decrease.data[0].y.max(),
#         text=f'Avg {stability_column_name_opposite}',
#         showarrow=True,
#         arrowhead=4,
#         arrowcolor='red',
#         ax=0,
#         ay=-40,
#         font=dict(color='red')
#     )
#     fig_decrease.add_annotation(
#         x=fig_decrease.data[0].x.max(),
#         y=avg_y_decrease,
#         text=f'Avg Normalized {capability_vector}',
#         showarrow=True,
#         arrowhead=4,
#         arrowcolor='blue',
#         ax=-40,
#         ay=0,
#         font=dict(color='blue')
#     )

#     # Add tooltips for -intervention
#     fig_decrease.update_traces(hoverinfo='text+name',
#                                hovertext=["District: " + str(d) +
#                                           f"<br>Impact ({capability_vector}): {y} <br>Stability ({base_column} -{intervention}%): {x}"
#                                           for x, y, d in zip(fig_decrease.data[0].x, fig_decrease.data[0].y, existing_data['District'])])

#     # Show the plots
#     fig_increase.show()
#     fig_decrease.show()

#     return existing_data

# base_Column = "TotalNPK_315"
# CapabilityVector = "Rice_Production_223"
# change_percentage = 20
# dim = 1

# # Call the function and get the handles for the two plots
# new_data = calc_and_vis_impact_stability_1D(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# # Display the resulting DataFrame
# new_data.head()


In [19]:

# base_Column = "TotalNPK_315"
# CapabilityVector = "Maize_Production_226"
# change_percentage = 20
# dim = 1

# # Call the function and get the handles for the two plots
# new_data = calc_and_vis_impact_stability_1D(G, existing_data.copy(), adjacency_file, base_Column, CapabilityVector, change_percentage, dim)

# # Display the resulting DataFrame
# new_data.head()

In [20]:
# Save the resulting DataFrame to a new CSV file if needed
# existing_data.to_csv('result.csv', index=False)

In [21]:
# from bokeh.models import Span, Label
# from bokeh.plotting import figure, show, output_notebook
# from bokeh.models.sources import ColumnDataSource
# from bokeh.models.tools import HoverTool
# import statsmodels.api as sm
# import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go

# def calc_and_vis_impact_stability_2D(G, existing_data, adjacency_file, base_column, capability_vector, intervention, dim):
#     # Calculate min and max values for changes made by each column in capability_vector
#     min_value_increase = float('inf')
#     max_value_increase = float('-inf')
#     min_value_decrease = float('inf')
#     max_value_decrease = float('-inf')
#     col_pass_plus = []
#     col_pass_minus = []
    
#     for cap_col in capability_vector:
#         # Calculate Impact Score
#         X = sm.add_constant(df[base_column])
#         y = df[cap_col]
#         model = sm.OLS(y, X).fit()

#         m, c = model.params[base_column], model.params['const']

#         base_column_increase = (1 + intervention/100) * df[base_column]
#         base_column_decrease = (1 - intervention/100) * df[base_column]

#         change_vector_increase = m * base_column_increase + c
#         change_vector_decrease = m * base_column_decrease + c
        
#         vector_plus = change_vector_increase - m * df[base_column] - c  
#         vector_minus = change_vector_decrease - m * df[base_column] - c

#         # Update min and max values for each column's changes
#         min_value_increase = min(min_value_increase, vector_plus.min())
#         max_value_increase = max(max_value_increase, vector_plus.max())

#         min_value_decrease = min(min_value_decrease, vector_minus.min())
#         max_value_decrease = max(max_value_decrease, vector_minus.max())

#     for cap_col in capability_vector:
#         # Calculate Impact Score
#         X = sm.add_constant(df[base_column])
#         y = df[cap_col]
#         model = sm.OLS(y, X).fit()

#         m, c = model.params[base_column], model.params['const']

#         base_column_increase = (1 + intervention/100) * df[base_column]
#         base_column_decrease = (1 - intervention/100) * df[base_column]

#         new_vector_increase = m * base_column_increase + c
#         new_vector_decrease = m * base_column_decrease + c
        
#         vector_plus = change_vector_increase - m * df[base_column] - c  
#         vector_minus = change_vector_decrease - m * df[base_column] - c

#         # Normalize changes manually using the calculated min and max values
#         scaler3 = MinMaxScaler()
#         normalized_new_increase_vector = scaler3.fit_transform(new_vector_increase.values.reshape(-1, 1))
    
#         scaler4 = MinMaxScaler()
#         normalized_new_decrease_vector = scaler4.fit_transform(new_vector_decrease.values.reshape(-1, 1))
        
#         normalized_change_increase_vector = (vector_plus - min_value_increase) / (max_value_increase - min_value_increase)
#         normalized_change_decrease_vector = (vector_minus - min_value_decrease) / (max_value_decrease - min_value_decrease)

#         result_df = pd.DataFrame({
#             f'{cap_col} ({base_column} +{intervention}%)': change_vector_increase,
#             f'{cap_col} ({base_column} -{intervention}%)': change_vector_decrease,
#             f'Normalized {capability_vector} ({base_column} +{intervention}%)': normalized_new_increase_vector.flatten(),
#             f'Normalized {capability_vector} ({base_column} -{intervention}%)': normalized_new_decrease_vector.flatten(),
#             f'Impact Score {cap_col} ({base_column} +{intervention}%)': normalized_change_increase_vector,
#             f'Impact Score {cap_col} ({base_column} -{intervention}%)': normalized_change_decrease_vector,
#         })

#         # Merge Columns
#         for column in result_df.columns:
#             if column in result_df.columns:
#                 existing_data[column] = result_df[column]

#         # Calculate Stability and add to DataFrame
#         columns_to_pass = f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"
#         col_pass_plus.append(columns_to_pass)

#         # Calculate Stability for the opposite intervention and add to DataFrame
#         opposite_intervention = -intervention
#         columns_to_pass_opposite = f"Normalized {cap_col} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"
#         col_pass_minus.append(columns_to_pass_opposite)
        
#     stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)

#     existing_data[stability_column_name] = stability_column

#     stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)

#     existing_data[stability_column_name_opposite] = stability_column_opposite
        

# # Visualize with Plotly for +intervention
#     fig_increase = go.Figure()
#     color_palette = ['red', 'blue', 'green', 'orange', 'purple']  # Add more colors if needed

#     for i, cap_col in enumerate(capability_vector):
#         # Scatter plot for +intervention
#         scatter_increase = go.Scatter(
#             x=existing_data[f"{stability_column_name}"],
#             y=existing_data[f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"],
#             mode='markers',
#             marker=dict(size=8, color=color_palette[i]),
#             name=f'{cap_col} +{intervention}%',
#             text=existing_data['District'],
#             hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
#         )

#         fig_increase.add_trace(scatter_increase)

#     # Update layout for +intervention plot
#     fig_increase.update_layout(
#         title=f'Impact vs Stability (+{intervention}%)',
#         xaxis=dict(title=f'Stability ({base_column} +{intervention}%)'),
#         yaxis=dict(title='Normalized Impact'),
#         hovermode='closest'
#     )

#     # Visualize with Plotly for -intervention
#     fig_decrease = go.Figure()

#     for i, cap_col in enumerate(capability_vector):
#         # Scatter plot for -intervention
#         scatter_decrease = go.Scatter(
#             x=existing_data[stability_column_name_opposite],
#             y=existing_data[f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"],
#             mode='markers',
#             marker=dict(size=8, color=color_palette[i]),
#             name=f'{cap_col} -{intervention}%',
#             text=existing_data['District'],
#             hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
#         )

#         fig_decrease.add_trace(scatter_decrease)

#     # Update layout for -intervention plot
#     fig_decrease.update_layout(
#         title=f'Impact vs Stability (-{intervention}%)',
#         xaxis=dict(title=f'Stability ({base_column} -{intervention}%)'),
#         yaxis=dict(title='Normalized Impact'),
#         hovermode='closest'
#     )

#     # Show plots
#     fig_increase.show()
#     fig_decrease.show()

#     return existing_data, fig_increase, fig_decrease

# # Assuming df is your DataFrame
# base_column = "TotalNPK_315"
# capability_vector = ["Rice_Production_223", "Jowar_Yield_278"]
# change_percentage = 20
# dim = 1

# # Call the function and get the handles for the two plots
# new_data, handle_increase, handle_decrease = calc_and_vis_impact_stability_2D(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentage, dim)

# # Display the resulting DataFrame
# new_data.head()


In [22]:
# new_data.head()

In [23]:
# new_data.to_csv('test.csv', index=False)

## 2D Capability vector

In [33]:
res_df = pd.DataFrame()

In [43]:
import statsmodels.api as sm
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

def calc_and_vis_impact_stability(G, existing_data, adjacency_file, base_column, capability_vector, intervention, dim):
    # Calculate min and max values for changes made by each column in capability_vector
    min_value_increase = float('inf')
    max_value_increase = float('-inf')
    min_value_decrease = float('inf')
    max_value_decrease = float('-inf')

    for cap_col in capability_vector:
        # Calculate Impact Score
        X = sm.add_constant(df[base_column])
        y = df[cap_col]
        model = sm.OLS(y, X).fit()

        m, c = model.params[base_column], model.params['const']

        base_column_increase = (1 + intervention/100) * df[base_column]
        base_column_decrease = (1 - intervention/100) * df[base_column]

        change_vector_increase = m * base_column_increase + c
        change_vector_decrease = m * base_column_decrease + c
        
        vector_plus = change_vector_increase - m * df[base_column] - c  
        vector_minus = change_vector_decrease - m * df[base_column] - c

        # Update min and max values for each column's changes
        min_value_increase = min(min_value_increase, vector_plus.min())
        max_value_increase = max(max_value_increase, vector_plus.max())

        min_value_decrease = min(min_value_decrease, vector_minus.min())
        max_value_decrease = max(max_value_decrease, vector_minus.max())

    # Normalize changes manually using the calculated min and max values
    col_pass_plus = []
    col_pass_minus = []
    for cap_col in capability_vector:
        # Calculate Impact Score
        X = sm.add_constant(df[base_column])
        y = df[cap_col]
        model = sm.OLS(y, X).fit()

        m, c = model.params[base_column], model.params['const']
        print(m,c)
        base_column_increase = (1 + intervention/100) * df[base_column]
        base_column_decrease = (1 - intervention/100) * df[base_column]

        new_vector_increase = m * base_column_increase + c
        new_vector_decrease = m * base_column_decrease + c
        
        vector_plus = new_vector_increase - m * df[base_column] - c  
        vector_minus = new_vector_decrease - m * df[base_column] - c


        # Normalize changes manually using the calculated min and max values
        normalized_change_increase_vector = (vector_plus - min_value_increase) / (max_value_increase - min_value_increase)
        normalized_change_decrease_vector = (vector_minus - min_value_decrease) / (max_value_decrease - min_value_decrease)
        
        
        # scaler = MinMaxScaler()
        # normalized_new_increase_vector = scaler.fit_transform(new_vector_increase.values.reshape(-1, 1))      
        # normalized_new_decrease_vector = scaler.fit_transform(new_vector_decrease.values.reshape(-1, 1))  
        
        # For new_vector_increase
        new_vector_increase_val = new_vector_increase.to_numpy().reshape(-1, 1)
        normalized_new_increase_vector = (new_vector_increase_val - new_vector_increase_val.min()) / (new_vector_increase_val.max() - new_vector_increase_val.min())

        # For new_vector_decrease
        new_vector_decrease_val = new_vector_decrease.to_numpy().reshape(-1, 1)
        normalized_new_decrease_vector = (new_vector_decrease_val - new_vector_decrease_val.min()) / (new_vector_decrease_val.max() - new_vector_decrease_val.min())
        
        result_df = pd.DataFrame({
            f'{cap_col} ({base_column} +{intervention}%)': new_vector_increase,
            f'{cap_col} ({base_column} -{intervention}%)': new_vector_decrease,
            f'Normalized {cap_col} ({base_column} +{intervention}%)': normalized_new_increase_vector.flatten(),
            f'Normalized {cap_col} ({base_column} -{intervention}%)': normalized_new_decrease_vector.flatten(),
            f'Impact Score {cap_col} ({base_column} +{intervention}%)': normalized_change_increase_vector,
            f'Impact Score {cap_col} ({base_column} -{intervention}%)': normalized_change_decrease_vector,
        })

        result_df_1 = pd.DataFrame({
            f'{cap_col} ({base_column} +{intervention}%)': new_vector_increase,
            # f'{cap_col} ({base_column} -{intervention}%)': new_vector_decrease,
            f'Normalized {cap_col} ({base_column} +{intervention}%)': normalized_new_increase_vector.flatten(),
            # f'Normalized {cap_col} ({base_column} -{intervention}%)': normalized_new_decrease_vector.flatten(),
            # f'Impact Score {cap_col} ({base_column} +{intervention}%)': normalized_change_increase_vector,
            # f'Impact Score {cap_col} ({base_column} -{intervention}%)': normalized_change_decrease_vector,
        })

        # Merge Columns
        for column in result_df.columns:
            existing_data[column] = result_df[column]
            
        for column in result_df_1.columns:
            res_df[column] = result_df_1[column]
        # Calculate Stability and add to DataFrame
        columns_to_pass = f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"
        col_pass_plus.append(columns_to_pass)
        
        # Calculate Stability for the opposite intervention and add to DataFrame
        opposite_intervention = -intervention
        columns_to_pass_opposite = f"Normalized {cap_col} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"
        col_pass_minus.append(columns_to_pass_opposite)

    
    # Normalize stability values
    stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)

    normalized_stability_vector = (stability_column - stability_column.min()) / (stability_column.max() - stability_column.min())
        
    capability_first_words = [col.split('_')[0] for col in capability_vector]
    stability_column_name_modified = stability_column_name + "_" + "_".join(capability_first_words)
    existing_data[stability_column_name_modified] = normalized_stability_vector
    
    stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)
        
    # Normalize stability values for the opposite intervention
    normalized_stability_vector_opposite = (stability_column_opposite - stability_column_opposite.min()) / (stability_column_opposite.max() - stability_column_opposite.min())    
    stability_column_name_opposite_modified = stability_column_name_opposite + "_" + "_".join(capability_first_words)
    existing_data[stability_column_name_opposite_modified] = normalized_stability_vector_opposite
    
    # Visualize with Plotly for +intervention
    fig_increase = go.Figure()
    color_palette = ['red', 'blue', 'green', 'orange', 'purple']  # Add more colors if needed

    for i, cap_col in enumerate(capability_vector):
        symbol = 'star' if i == 0 else 'circle'  # Use 'star' for the first point, 'circle' for others
        scatter_increase = go.Scatter(
            x=existing_data[stability_column_name_modified],
            y=existing_data[f"Impact Score {cap_col} ({base_column} +{abs(intervention)}%)"],
            mode='markers',
            marker=dict(size=7, symbol=symbol, color=color_palette[i]),  # Adjust the size and add symbol parameter
            name=f'{cap_col} +{intervention}%',
            text=existing_data['District'],
            hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
        )
        fig_increase.add_trace(scatter_increase)

    # Add average lines
    avg_stability_increase = existing_data[stability_column_name_modified].mean()
    avg_impact_increase = existing_data[f"Impact Score {capability_vector[0]} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"].mean()

    fig_increase.add_shape(
        go.layout.Shape(
            type="line",
            x0=avg_stability_increase,
            x1=avg_stability_increase,
            y0=0,
            y1=1,
            line=dict(color="black", dash="dash"),
            name="Average Stability"
        )
    )
    fig_increase.add_shape(
        go.layout.Shape(
            type="line",
            x0=0,
            x1=1,
            y0=avg_impact_increase,
            y1=avg_impact_increase,
            line=dict(color="black", dash="dash"),
            name="Average Impact"
        )
    )

    # Update layout for +intervention plot
    fig_increase.update_layout(
        title=f'Impact vs Stability (+{intervention}%)',
        xaxis=dict(title=f'Stability ({base_column} +{intervention}%)'),
        yaxis=dict(title='Normalized Impact'),
        hovermode='closest'
    )

    # Visualize with Plotly for -intervention
    fig_decrease = go.Figure()

    for i, cap_col in enumerate(capability_vector):
        # Scatter plot for -intervention
        symbol = 'star' if i == 0 else 'circle'
        scatter_decrease = go.Scatter(
            x=existing_data[stability_column_name_opposite_modified],
            y=existing_data[f"Impact Score {cap_col} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"],
            mode='markers',
            marker=dict(size=7,symbol=symbol, color=color_palette[i]),
            name=f'{cap_col} -{intervention}%',
            text=existing_data['District'],
            hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
        )

        fig_decrease.add_trace(scatter_decrease)

    # Add average lines for -intervention
    avg_stability_decrease = existing_data[stability_column_name_opposite_modified].mean()
    avg_impact_decrease = existing_data[f"Impact Score {capability_vector[0]} ({base_column} {'+' if intervention <= 0 else '-'}{abs(intervention)}%)"].mean()

    fig_decrease.add_shape(
        go.layout.Shape(
            type="line",
            x0=avg_stability_decrease,
            x1=avg_stability_decrease,
            y0=0,
            y1=1,
            line=dict(color="black", dash="dash"),
            name="Average Stability"
        )
    )
    fig_decrease.add_shape(
        go.layout.Shape(
            type="line",
            x0=0,
            x1=1,
            y0=avg_impact_decrease,
            y1=avg_impact_decrease,
            line=dict(color="black", dash="dash"),
            name="Average Impact"
        )
    )

    # Update layout for -intervention plot
    fig_decrease.update_layout(
        title=f'Impact vs Stability (-{intervention}%)',
        xaxis=dict(title=f'Stability ({base_column} -{intervention}%)'),
        yaxis=dict(title='Normalized Impact'),
        hovermode='closest'
    )

    # Show plots
    fig_increase.show()
    fig_decrease.show()

    return existing_data, fig_increase, fig_decrease, result_df

base_column = "TotalNPK_315"
capability_vector = ["Rice_Production_223", "Jowar_Yield_278"]
change_percentage = 20
dim = 2

new_data, handle_increase, handle_decrease,result_df = calc_and_vis_impact_stability(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentage, dim)
new_data.head()

2.353363480056113 8136.426218911565
0.009814190872938423 364.522399314813


,District,Rice_Production_223 (TotalNPK_315 +20%),Rice_Production_223 (TotalNPK_315 -20%),Normalized Rice_Production_223 (TotalNPK_315 +20%),Normalized Rice_Production_223 (TotalNPK_315 -20%),Impact Score Rice_Production_223 (TotalNPK_315 +20%),Impact Score Rice_Production_223 (TotalNPK_315 -20%),Jowar_Yield_278 (TotalNPK_315 +20%),Jowar_Yield_278 (TotalNPK_315 -20%),Normalized Jowar_Yield_278 (TotalNPK_315 +20%),Normalized Jowar_Yield_278 (TotalNPK_315 -20%),Impact Score Jowar_Yield_278 (TotalNPK_315 +20%),Impact Score Jowar_Yield_278 (TotalNPK_315 -20%),New Stability(NPK + 20%)_Rice_Jowar,New Stability(NPK - 20%)_Rice_Jowar
0,BENGALURU,73964.709483,52021.948395,0.098721,0.098721,0.129382,0.870618,639.044946,547.537431,0.098721,0.098721,0.000398,0.999602,0.959044,0.959044
1,BENGALURU(R),62524.538934,44395.168029,0.075418,0.075418,0.106872,0.893128,591.336202,515.731601,0.075418,0.075418,0.000304,0.999696,0.976505,0.976505
2,RAMANAGARA,29237.624526,22203.891757,0.007616,0.007616,0.041376,0.958624,452.520360,423.187707,0.007616,0.007616,0.000031,0.999969,0.822250,0.822250
3,CHITRADURGA,109474.140361,75694.902314,0.171050,0.171050,0.199250,0.800750,787.129310,646.260340,0.171050,0.171050,0.000689,0.999311,0.668650,0.668650
4,DAVANAGERE,289534.686947,195735.266704,0.537816,0.537816,0.553539,0.446461,1538.032682,1146.862588,0.537816,0.537816,0.002167,0.997833,0.790058,0.790058


In [25]:
# import statsmodels.api as sm
# import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go

# def calc_and_vis_impact_stability(G, existing_data, adjacency_file, base_column, capability_vector, intervention, dim):
#     # Calculate min and max values for changes made by each column in capability_vector
#     min_value_increase = float('inf')
#     max_value_increase = float('-inf')
#     min_value_decrease = float('inf')
#     max_value_decrease = float('-inf')

#     for cap_col in capability_vector:
#         # Calculate Impact Score
#         X = sm.add_constant(df[base_column])
#         y = df[cap_col]
#         model = sm.OLS(y, X).fit()

#         m, c = model.params[base_column], model.params['const']

#         base_column_increase = (1 + intervention/100) * df[base_column]
#         base_column_decrease = (1 - intervention/100) * df[base_column]

#         change_vector_increase = m * base_column_increase + c
#         change_vector_decrease = m * base_column_decrease + c
        
#         vector_plus = change_vector_increase - m * df[base_column] - c  
#         vector_minus = change_vector_decrease - m * df[base_column] - c

#         # Update min and max values for each column's changes
#         min_value_increase = min(min_value_increase, vector_plus.min())
#         max_value_increase = max(max_value_increase, vector_plus.max())

#         min_value_decrease = min(min_value_decrease, vector_minus.min())
#         max_value_decrease = max(max_value_decrease, vector_minus.max())

#     # Normalize changes manually using the calculated min and max values
#     col_pass_plus = []
#     col_pass_minus = []
#     for cap_col in capability_vector:
#         # Calculate Impact Score
#         X = sm.add_constant(df[base_column])
#         y = df[cap_col]
#         model = sm.OLS(y, X).fit()

#         m, c = model.params[base_column], model.params['const']

#         base_column_increase = (1 + intervention/100) * df[base_column]
#         base_column_decrease = (1 - intervention/100) * df[base_column]

#         new_vector_increase = m * base_column_increase + c
#         new_vector_decrease = m * base_column_decrease + c
        
#         vector_plus = new_vector_increase - m * df[base_column] - c  
#         vector_minus = new_vector_decrease - m * df[base_column] - c

#         # Normalize changes manually using the calculated min and max values
#         normalized_change_increase_vector = (vector_plus - min_value_increase) / (max_value_increase - min_value_increase)
#         normalized_change_decrease_vector = (vector_minus - min_value_decrease) / (max_value_decrease - min_value_decrease)

#         scaler = MinMaxScaler()
#         normalized_new_increase_vector = scaler.fit_transform(new_vector_increase.values.reshape(-1, 1))
#         normalized_new_decrease_vector = scaler.fit_transform(new_vector_decrease.values.reshape(-1, 1))
        
#         result_df = pd.DataFrame({
#             f'{cap_col} ({base_column} +{intervention}%)': new_vector_increase,
#             f'{cap_col} ({base_column} -{intervention}%)': new_vector_decrease,
#             f'Normalized {cap_col} ({base_column} +{intervention}%)': normalized_new_increase_vector,
#             f'Normalized {cap_col} ({base_column} -{intervention}%)': normalized_new_decrease_vector,
#             f'Impact Score {cap_col} ({base_column} +{intervention}%)': normalized_change_increase_vector,
#             f'Impact Score {cap_col} ({base_column} -{intervention}%)': normalized_change_decrease_vector,

#         })

#         # Merge Columns
#         for column in result_df.columns:
#             if column in result_df.columns:
#                 existing_data[column] = result_df[column]

#         # Calculate Stability and add to DataFrame
#         columns_to_pass = f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"
#         col_pass_plus.append(columns_to_pass)
        
#         # Calculate Stability for the opposite intervention and add to DataFrame
#         opposite_intervention = -intervention
#         columns_to_pass_opposite = f"Normalized {cap_col} ({base_column} {'+' if opposite_intervention >= 0 else '-'}{abs(opposite_intervention)}%)"
#         col_pass_minus.append(columns_to_pass_opposite)
        
#     # Normalize stability values
#     stability_column, stability_column_name = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_plus, intervention, dim)

#     normalized_stability_vector = (stability_column - stability_column.min()) / (stability_column.max() - stability_column.min())
        
#     capability_first_words = [col.split('_')[0] for col in capability_vector]
#     stability_column_name_modified = stability_column_name + "_" + "_".join(capability_first_words)
#     existing_data[stability_column_name_modified] = normalized_stability_vector
    
#     stability_column_opposite, stability_column_name_opposite = calculate_and_map_stability(G, existing_data.copy(), adjacency_file, col_pass_minus, opposite_intervention, dim)
        
#     # Normalize stability values for the opposite intervention
#     normalized_stability_vector_opposite = (stability_column_opposite - stability_column_opposite.min()) / (stability_column_opposite.max() - stability_column_opposite.min())    
#     stability_column_name_opposite_modified = stability_column_name_opposite + "_" + "_".join(capability_first_words)
#     existing_data[stability_column_name_opposite_modified] = normalized_stability_vector_opposite
    
#     # Visualize with Plotly for +intervention
#     fig_increase = go.Figure()
#     color_palette = ['red', 'blue', 'green', 'orange', 'purple']  # Add more colors if needed

#     for i, cap_col in enumerate(capability_vector):
#         symbol = 'star' if i == 0 else 'circle'  # Use 'star' for the first point, 'circle' for others
#         scatter_increase = go.Scatter(
#             x=existing_data[stability_column_name_modified],
#             y=existing_data[f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"],
#             mode='markers',
#             marker=dict(size=7, symbol=symbol, color=color_palette[i]),  # Adjust the size and add symbol parameter
#             name=f'{cap_col} +{intervention}%',
#             text=existing_data['District'],
#             hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
#         )
#         fig_increase.add_trace(scatter_increase)

#     # Add average lines
#     avg_stability_increase = existing_data[stability_column_name_modified].mean()
#     avg_impact_increase = existing_data[f"Normalized {capability_vector[0]} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"].mean()

#     fig_increase.add_shape(
#         go.layout.Shape(
#             type="line",
#             x0=avg_stability_increase,
#             x1=avg_stability_increase,
#             y0=0,
#             y1=1,
#             line=dict(color="black", dash="dash"),
#             name="Average Stability"
#         )
#     )
#     fig_increase.add_shape(
#         go.layout.Shape(
#             type="line",
#             x0=0,
#             x1=1,
#             y0=avg_impact_increase,
#             y1=avg_impact_increase,
#             line=dict(color="black", dash="dash"),
#             name="Average Impact"
#         )
#     )

#     # Update layout for +intervention plot
#     fig_increase.update_layout(
#         title=f'Impact vs Stability (+{intervention}%)',
#         xaxis=dict(title=f'Stability ({base_column} +{intervention}%)'),
#         yaxis=dict(title='Normalized Impact'),
#         hovermode='closest'
#     )

#     # Visualize with Plotly for -intervention
#     fig_decrease = go.Figure()

#     for i, cap_col in enumerate(capability_vector):
#         # Scatter plot for -intervention
#         symbol = 'star' if i == 0 else 'circle'
#         scatter_decrease = go.Scatter(
#             x=existing_data[stability_column_name_opposite_modified],
#             y=existing_data[f"Normalized {cap_col} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"],
#             mode='markers',
#             marker=dict(size=7,symbol=symbol, color=color_palette[i]),
#             name=f'{cap_col} -{intervention}%',
#             text=existing_data['District'],
#             hovertemplate='%{text}' + '<br>Impact: %{y:.2f}' + '<br>Stability: %{x:.2f}'
#         )

#         fig_decrease.add_trace(scatter_decrease)

#     # Add average lines for -intervention
#     avg_stability_decrease = existing_data[stability_column_name_opposite_modified].mean()
#     avg_impact_decrease = existing_data[f"Normalized {capability_vector[0]} ({base_column} {'+' if intervention >= 0 else '-'}{abs(intervention)}%)"].mean()

#     fig_decrease.add_shape(
#         go.layout.Shape(
#             type="line",
#             x0=avg_stability_decrease,
#             x1=avg_stability_decrease,
#             y0=0,
#             y1=1,
#             line=dict(color="black", dash="dash"),
#             name="Average Stability"
#         )
#     )
#     fig_decrease.add_shape(
#         go.layout.Shape(
#             type="line",
#             x0=0,
#             x1=1,
#             y0=avg_impact_decrease,
#             y1=avg_impact_decrease,
#             line=dict(color="black", dash="dash"),
#             name="Average Impact"
#         )
#     )

#     # Update layout for -intervention plot
#     fig_decrease.update_layout(
#         title=f'Impact vs Stability (-{intervention}%)',
#         xaxis=dict(title=f'Stability ({base_column} -{intervention}%)'),
#         yaxis=dict(title='Normalized Impact'),
#         hovermode='closest'
#     )

#     # Show plots
#     fig_increase.show()
#     fig_decrease.show()

#     return existing_data, fig_increase, fig_decrease

# base_column = "TotalNPK_315"
# capability_vector = ["Rice_Production_223", "Jowar_Yield_278"]
# change_percentage = 20
# dim = 2

# new_data, handle_increase, handle_decrease = calc_and_vis_impact_stability(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentage, dim)
# new_data.head()

In [41]:
new_data

,District,Rice_Production_223 (TotalNPK_315 +20%),Rice_Production_223 (TotalNPK_315 -20%),Normalized Rice_Production_223 (TotalNPK_315 +20%),Normalized Rice_Production_223 (TotalNPK_315 -20%),Impact Score Rice_Production_223 (TotalNPK_315 +20%),Impact Score Rice_Production_223 (TotalNPK_315 -20%),Jowar_Yield_278 (TotalNPK_315 +20%),Jowar_Yield_278 (TotalNPK_315 -20%),Normalized Jowar_Yield_278 (TotalNPK_315 +20%),Normalized Jowar_Yield_278 (TotalNPK_315 -20%),Impact Score Jowar_Yield_278 (TotalNPK_315 +20%),Impact Score Jowar_Yield_278 (TotalNPK_315 -20%),New Stability(NPK + 20%)_Rice_Jowar,New Stability(NPK - 20%)_Rice_Jowar
0,BENGALURU,73964.709483,52021.948395,0.098721,0.098721,0.129382,0.870618,639.044946,547.537431,0.098721,0.098721,0.000398,0.999602,0.959044,0.959044
1,BENGALURU(R),62524.538934,44395.168029,0.075418,0.075418,0.106872,0.893128,591.336202,515.731601,0.075418,0.075418,0.000304,0.999696,0.976505,0.976505
2,RAMANAGARA,29237.624526,22203.891757,0.007616,0.007616,0.041376,0.958624,452.520360,423.187707,0.007616,0.007616,0.000031,0.999969,0.822250,0.822250
3,CHITRADURGA,109474.140361,75694.902314,0.171050,0.171050,0.199250,0.800750,787.129310,646.260340,0.171050,0.171050,0.000689,0.999311,0.668650,0.668650
4,DAVANAGERE,289534.686947,195735.266704,0.537816,0.537816,0.553539,0.446461,1538.032682,1146.862588,0.537816,0.537816,0.002167,0.997833,0.790058,0.790058
5,KOLAR,70801.788966,49913.334717,0.092278,0.092278,0.123158,0.876842,625.854674,538.743916,0.092278,0.092278,0.000372,0.999628,1.000000,1.000000
6,CHIKKABALLAPURA,88217.620064,61523.888782,0.127752,0.127752,0.157426,0.842574,698.483612,587.163208,0.127752,0.127752,0.000515,0.999485,0.997250,0.997250
7,SHIVAMOGGA,168002.290110,114713.668813,0.290266,0.290266,0.314411,0.685589,1031.208237,808.979624,0.290266,0.290266,0.001169,0.998831,0.967812,0.967812
8,TUMAKURU,115867.758264,79957.314249,0.184073,0.184073,0.211830,0.788170,813.792503,664.035802,0.184073,0.184073,0.000742,0.999258,0.988802,0.988802
9,CHIKKAMAGALURU,199783.993235,135901.470897,0.355002,0.355002,0.376945,0.623055,1163.746922,897.338747,0.355002,0.355002,0.001430,0.998570,0.850936,0.850936


In [35]:
res_df.head()

,Rice_Production_223 (TotalNPK_315 +20%),Normalized Rice_Production_223 (TotalNPK_315 +20%),Jowar_Yield_278 (TotalNPK_315 +20%),Normalized Jowar_Yield_278 (TotalNPK_315 +20%)
0,73964.709483,0.098721,639.044946,0.098721
1,62524.538934,0.075418,591.336202,0.075418
2,29237.624526,0.007616,452.520360,0.007616
3,109474.140361,0.171050,787.129310,0.171050
4,289534.686947,0.537816,1538.032682,0.537816


In [37]:
print(res_df['Rice_Production_223 (TotalNPK_315 +20%)'].min(), res_df['Rice_Production_223 (TotalNPK_315 +20%)'].max())

25498.600629373545 516440.3456216234


In [38]:
print(res_df['Jowar_Yield_278 (TotalNPK_315 +20%)'].min(), res_df['Jowar_Yield_278 (TotalNPK_315 +20%)'].max())

436.9275738990035 2484.293411637132


In [39]:
res_df

,Rice_Production_223 (TotalNPK_315 +20%),Normalized Rice_Production_223 (TotalNPK_315 +20%),Jowar_Yield_278 (TotalNPK_315 +20%),Normalized Jowar_Yield_278 (TotalNPK_315 +20%)
0,73964.709483,0.098721,639.044946,0.098721
1,62524.538934,0.075418,591.336202,0.075418
2,29237.624526,0.007616,452.520360,0.007616
3,109474.140361,0.171050,787.129310,0.171050
4,289534.686947,0.537816,1538.032682,0.537816
5,70801.788966,0.092278,625.854674,0.092278
6,88217.620064,0.127752,698.483612,0.127752
7,168002.290110,0.290266,1031.208237,0.290266
8,115867.758264,0.184073,813.792503,0.184073
9,199783.993235,0.355002,1163.746922,0.355002


In [26]:
new_data.head()

,District,Rice_Production_223 (TotalNPK_315 +20%),Rice_Production_223 (TotalNPK_315 -20%),Normalized Rice_Production_223 (TotalNPK_315 +20%),Normalized Rice_Production_223 (TotalNPK_315 -20%),Impact Score Rice_Production_223 (TotalNPK_315 +20%),Impact Score Rice_Production_223 (TotalNPK_315 -20%),Jowar_Yield_278 (TotalNPK_315 +20%),Jowar_Yield_278 (TotalNPK_315 -20%),Normalized Jowar_Yield_278 (TotalNPK_315 +20%),Normalized Jowar_Yield_278 (TotalNPK_315 -20%),Impact Score Jowar_Yield_278 (TotalNPK_315 +20%),Impact Score Jowar_Yield_278 (TotalNPK_315 -20%),New Stability(NPK + 20%)_Rice_Jowar,New Stability(NPK - 20%)_Rice_Jowar
0,BENGALURU,73964.709483,52021.948395,0.098721,0.098721,0.129382,0.870618,639.044946,547.537431,0.098721,0.098721,0.000398,0.999602,0.959044,0.959044
1,BENGALURU(R),62524.538934,44395.168029,0.075418,0.075418,0.106872,0.893128,591.336202,515.731601,0.075418,0.075418,0.000304,0.999696,0.976505,0.976505
2,RAMANAGARA,29237.624526,22203.891757,0.007616,0.007616,0.041376,0.958624,452.520360,423.187707,0.007616,0.007616,0.000031,0.999969,0.822250,0.822250
3,CHITRADURGA,109474.140361,75694.902314,0.171050,0.171050,0.199250,0.800750,787.129310,646.260340,0.171050,0.171050,0.000689,0.999311,0.668650,0.668650
4,DAVANAGERE,289534.686947,195735.266704,0.537816,0.537816,0.553539,0.446461,1538.032682,1146.862588,0.537816,0.537816,0.002167,0.997833,0.790058,0.790058


In [45]:
base_column = "TotalNPK_315"
capability_vector = ["Maize_Production_226", "Rice_Production_223"]
change_percentage = 20
dim = 1

new_data, handle_increase, handle_decrease,result_df = calc_and_vis_impact_stability(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentage, dim)
new_data.head()

2.564730341506419 562.38192631683
2.353363480056113 8136.426218911565


,District,Maize_Production_226 (TotalNPK_315 +20%),Maize_Production_226 (TotalNPK_315 -20%),Normalized Maize_Production_226 (TotalNPK_315 +20%),Normalized Maize_Production_226 (TotalNPK_315 -20%),Impact Score Maize_Production_226 (TotalNPK_315 +20%),Impact Score Maize_Production_226 (TotalNPK_315 -20%),Rice_Production_223 (TotalNPK_315 +20%),Rice_Production_223 (TotalNPK_315 -20%),Normalized Rice_Production_223 (TotalNPK_315 +20%),Normalized Rice_Production_223 (TotalNPK_315 -20%),Impact Score Rice_Production_223 (TotalNPK_315 +20%),Impact Score Rice_Production_223 (TotalNPK_315 -20%),New Stability(NPK + 20%)_Maize_Rice,New Stability(NPK - 20%)_Maize_Rice
0,BENGALURU,72303.019039,48389.473335,0.098721,0.098721,0.101340,0.898660,73964.709483,52021.948395,0.098721,0.098721,0.090322,0.909678,0.959044,0.959044
1,BENGALURU(R),59835.351903,40077.695244,0.075418,0.075418,0.078105,0.921895,62524.538934,44395.168029,0.075418,0.075418,0.069002,0.930998,0.976505,0.976505
2,RAMANAGARA,23558.780060,15893.314016,0.007616,0.007616,0.010500,0.989500,29237.624526,22203.891757,0.007616,0.007616,0.006968,0.993032,0.822250,0.822250
3,CHITRADURGA,111001.722216,74188.608786,0.171050,0.171050,0.173459,0.826541,109474.140361,75694.902314,0.171050,0.171050,0.156497,0.843503,0.668650,0.668650
4,DAVANAGERE,307234.370105,205010.374046,0.537816,0.537816,0.539159,0.460841,289534.686947,195735.266704,0.537816,0.537816,0.492058,0.507942,0.790058,0.790058


In [ ]:
base_column = "TotalNPK_315"
capability_vector = ["Maize_Production_226", "Rice_Production_223"]
change_percentage = 10
dim = 2

new_data, handle_increase, handle_decrease = calc_and_vis_impact_stability(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentage, dim)
new_data.head()

,District,Maize_Production_226 (TotalNPK_315 +10%),Maize_Production_226 (TotalNPK_315 -10%),Normalized Maize_Production_226 (TotalNPK_315 +10%),Normalized Maize_Production_226 (TotalNPK_315 -10%),Impact Score Maize_Production_226 (TotalNPK_315 +10%),Impact Score Maize_Production_226 (TotalNPK_315 -10%),Rice_Production_223 (TotalNPK_315 +10%),Rice_Production_223 (TotalNPK_315 -10%),Normalized Rice_Production_223 (TotalNPK_315 +10%),Normalized Rice_Production_223 (TotalNPK_315 -10%),Impact Score Rice_Production_223 (TotalNPK_315 +10%),Impact Score Rice_Production_223 (TotalNPK_315 -10%),New Stability(NPK + 10%)_Maize_Rice,New Stability(NPK - 10%)_Maize_Rice
0,BENGALURU,66324.632613,54367.859761,0.098721,0.098721,0.101340,0.898660,68479.019211,57507.638667,0.098721,0.098721,0.090322,0.909678,0.959044,0.959044
1,BENGALURU(R),54895.937738,45017.109409,0.075418,0.075418,0.078105,0.921895,57992.196208,48927.510755,0.075418,0.075418,0.069002,0.930998,0.976505,0.976505
2,RAMANAGARA,21642.413549,17809.680527,0.007616,0.007616,0.010500,0.989500,27479.191334,23962.324950,0.007616,0.007616,0.006968,0.993032,0.822250,0.822250
3,CHITRADURGA,101798.443858,83391.887143,0.171050,0.171050,0.173459,0.826541,101029.330849,84139.711825,0.171050,0.171050,0.156497,0.843503,0.668650,0.668650
4,DAVANAGERE,281678.371090,230566.373060,0.537816,0.537816,0.539159,0.460841,266084.831886,219185.121765,0.537816,0.537816,0.492058,0.507942,0.790058,0.790058


In [48]:
base_column = "TotalNPK_315"
capability_vector = ["Maize_Production_226", "Rice_Production_223","Jowar_Yield_278"]
change_percentage = 20
dim = 3

new_data_3d, handle_increase, handle_decrease,result_df = calc_and_vis_impact_stability(G, existing_data.copy(), adjacency_file, base_column, capability_vector, change_percentage, dim)
new_data_3d.head()

2.564730341506419 562.38192631683
2.353363480056113 8136.426218911565
0.009814190872938423 364.522399314813


,District,Maize_Production_226 (TotalNPK_315 +20%),Maize_Production_226 (TotalNPK_315 -20%),Normalized Maize_Production_226 (TotalNPK_315 +20%),Normalized Maize_Production_226 (TotalNPK_315 -20%),Impact Score Maize_Production_226 (TotalNPK_315 +20%),Impact Score Maize_Production_226 (TotalNPK_315 -20%),Rice_Production_223 (TotalNPK_315 +20%),Rice_Production_223 (TotalNPK_315 -20%),Normalized Rice_Production_223 (TotalNPK_315 +20%),...,Impact Score Rice_Production_223 (TotalNPK_315 +20%),Impact Score Rice_Production_223 (TotalNPK_315 -20%),Jowar_Yield_278 (TotalNPK_315 +20%),Jowar_Yield_278 (TotalNPK_315 -20%),Normalized Jowar_Yield_278 (TotalNPK_315 +20%),Normalized Jowar_Yield_278 (TotalNPK_315 -20%),Impact Score Jowar_Yield_278 (TotalNPK_315 +20%),Impact Score Jowar_Yield_278 (TotalNPK_315 -20%),New Stability(NPK + 20%)_Maize_Rice_Jowar,New Stability(NPK - 20%)_Maize_Rice_Jowar
0,BENGALURU,72303.019039,48389.473335,0.098721,0.098721,0.129392,0.870608,73964.709483,52021.948395,0.098721,...,0.118718,0.881282,639.044946,547.537431,0.098721,0.098721,0.000365,0.999635,0.959044,0.959044
1,BENGALURU(R),59835.351903,40077.695244,0.075418,0.075418,0.106882,0.893118,62524.538934,44395.168029,0.075418,...,0.098063,0.901937,591.336202,515.731601,0.075418,0.075418,0.000279,0.999721,0.976505,0.976505
2,RAMANAGARA,23558.780060,15893.314016,0.007616,0.007616,0.041388,0.958612,29237.624526,22203.891757,0.007616,...,0.037966,0.962034,452.520360,423.187707,0.007616,0.007616,0.000028,0.999972,0.822250,0.822250
3,CHITRADURGA,111001.722216,74188.608786,0.171050,0.171050,0.199260,0.800740,109474.140361,75694.902314,0.171050,...,0.182827,0.817173,787.129310,646.260340,0.171050,0.171050,0.000632,0.999368,0.668650,0.668650
4,DAVANAGERE,307234.370105,205010.374046,0.537816,0.537816,0.553544,0.446456,289534.686947,195735.266704,0.537816,...,0.507914,0.492086,1538.032682,1146.862588,0.537816,0.537816,0.001988,0.998012,0.790058,0.790058


In [ ]:
# List of districts in the dataset
districts = df['District'].tolist()

# List of columns to visualize
columns_to_visualize = ['Rice_Production_223', 'Jowar_Yield_278', 'Maize_Production_226', 'SowingSeedsDistributed_Jowar_318', 'SowingSeedsDistributed_Maize_320']

colors = Category20[len(columns_to_visualize)]

# Create a figure for each column
for column, color in zip(columns_to_visualize, colors):
    # Create a new plot
    plot = figure(title=f'{column} vs District', x_range=districts, sizing_mode='stretch_width', toolbar_location=None, tools='')

    # Plot the data
    source = ColumnDataSource(data=dict(x=districts, y=df[column]))
    plot.vbar(x='x', top='y', width=0.8, color=color, legend_label=column, source=source)

    # Add hover tooltips
    hover = HoverTool()
    hover.tooltips = [('District', '@x'), (column, '@y')]
    plot.add_tools(hover)

    # Styling
    plot.xaxis.major_label_orientation = "vertical"
    plot.yaxis.axis_label = column
    plot.legend.location = 'top_right'
    plot.legend.click_policy = 'hide'

    # Show the plot
    show(plot)


In [ ]:
from bokeh.models import FactorRange
from bokeh.transform import factor_cmap

# Assuming df is your DataFrame

# List of crops
crops = ['Rice_Production_223', 'Jowar_Yield_278', 'Maize_Production_226']

# Create a new plot
plot = figure(title='Production Comparison by Crop and District', x_range=FactorRange(*districts),toolbar_location=None, tools='')

# Plot the data
source = ColumnDataSource(df)
plot.vbar_stack(crops, x='District', width=0.9, color=Category20[len(crops)], source=source, legend_label=crops)

# Add hover tooltips
hover = HoverTool()
hover.tooltips = [('District', '@District')] + [(crop, f'@{crop}') for crop in crops]
plot.add_tools(hover)

# Styling
plot.xaxis.major_label_orientation = "vertical"
plot.yaxis.axis_label = "Production"
plot.legend.location = 'top_left'
plot.legend.click_policy = 'hide'

# Show the plot
show(plot)
